# Start

In [4]:
import numpy as np
import matplotlib.pyplot as plt

import panel as pn
pn.extension("ipywidgets", 'katex', 'mathjax')
import ipywidgets as ipw
from matplotlib.animation import FuncAnimation
from matplotlib.ticker import MultipleLocator

from random import shuffle, uniform

print("Packages succesfully loaded")

Packages succesfully loaded


# Functions

In [5]:
def limit_answer(x):
    s = str(x)

    # inspired by: https://stackoverflow.com/questions/35585950/find-the-number-of-digits-after-the-decimal-point
    if not '.' in s:
        n_decimal = 0
    else:
        n_decimal = len(s) - s.index('.') - 1

    range = 5*10**-(n_decimal+1)

    return range

def check_nummeric_answers(id, answer, unit, FB_G, FB_W, num_widget, feedback_widget, attempt):
    
    def button_callback(b):
        attempt.value += 1
        #print("debug question:", id, 'attempt', attempt.value ,', response:' ,num_widget.value, ', answer:',answer)

        # the answer is within the boundaries, print positive feedback
        if np.abs(answer - num_widget.value) < limit_answer(answer):
            if len(FB_G) != 0:
                feedback_widget.value = FB_G
            else:
                feedback_widget.value = 'Well done, this is correct!'

        # the answer is NOT within boundaries, provide feedback based on the number of attempts
        if np.abs(answer - num_widget.value) >= limit_answer(answer):

            if attempt.value < 3 and len(FB_W) > 0:
                feedback_widget.value = FB_W
                
            if attempt.value < 3 and len(FB_W) == 0:
                feedback_widget.value = 'Oops, there seems to be a mistake'
                
            if attempt.value >= 3:
                feedback_widget.value = 'The correct answer is ' + str(answer) + str(unit) + '.'

    return button_callback  # otherwise gives TypeError: 'NoneType' object is not callable

def nummeric_question_body(questions, units, answers, FB_good, FB_wrong):
    all_widgets = []
    attempts = []
    id = 0
    for question, units, answer, Q_FB_G, Q_FB_W in zip(questions, units, answers, FB_good, FB_wrong):
        id += 1
        question_widget = pn.widgets.StaticText(value=question, width = 750)
        unit_widget = pn.widgets.StaticText(value=units, width = 10)
        num_widget = pn.widgets.FloatInput(value=0, step=0.01, width = 100)
        #feedback_widget = pn.widgets.TextInput(value="", name="", width=500)
        feedback_widget = pn.widgets.StaticText(value="", name="", width=500)
        submit_button =  pn.widgets.Button(name="Check")
        
        Hbox = pn.Row(num_widget, unit_widget, submit_button, feedback_widget)       
        quiz_widget = pn.Column(question_widget, Hbox)

        all_widgets.append(quiz_widget)

        # the values for the submit button are determined at the moment these are created.
        attempt = pn.widgets.FloatInput(value=0)
        attempts.append(attempt)
        submit_button.on_click(check_nummeric_answers(id, answer, units, Q_FB_G, Q_FB_W, num_widget, feedback_widget, attempt))
        
    return all_widgets

In [6]:
def dispersion(k, h):  # calculate omega
    return (9.81 * k * np.tanh(k * h)) ** 0.5

In [7]:
def wave_length(T, h):
    L = 9.81 * T**2 / (2 * np.pi)
    L_all = [L]

    for i in range(1500):
        L = 9.81 * T**2 / (2 * np.pi) * np.tanh(2 * np.pi * h / L)
        L_all.append(L)

        if np.abs(L_all[-1] - L_all[-2]) < 0.0005:
            #plt.plot(L_all)
            break
            
    return round(L, 13)

#wave_length(T = 3, h = 600)

In [8]:
def wave_length_check(T,h):
    Dif = []
    L_serie = np.arange(5,1000,0.01) 
    for L in L_serie:
        LHS = (2*np.pi/T)**2
        RHS = 9.81*2*np.pi/L * np.tanh(2*np.pi/L * h)
        Dif.append(np.abs(LHS-RHS))
        #if len(Dif) > 2 and Dif[-1] > Dif[-2]:
        #    break
        
    id_best = np.argmin(Dif)
    L= L_serie[id_best]
    plt.plot(Dif)
    
    return round(L,13)

#wave_length_check(T = 3,h = 600)

## Student calculators

In [9]:
def wavelength_calculator():
    # define widgets (with initial value for L)
    T = pn.widgets.FloatInput(value=5, step=0.01, width=100)
    h = pn.widgets.FloatInput(value=3, step=0.01, width=100)
    
    L_init = wave_length(T.value, h.value)
    L = pn.widgets.FloatInput(value=L_init, step=0.001, width= 100, disabled=True)

    # change the value of L when a change in T or h is observed
    def update_L_widget(event):
        L.value = wave_length(T.value, h.value)

    T.param.watch(update_L_widget, 'value')
    h.param.watch(update_L_widget, 'value')

    # set the surrounding layout, like headings and descriptions
    ## set headings and the column width
    title_input = pn.widgets.StaticText(value='<b>Input</b>', width=200)
    title_output = pn.widgets.StaticText(value='<b>Output</b>', width=200)
    heading = pn.widgets.StaticText(value='<b>Wavelength calculator</b>')
    heading = pn.widgets.StaticText(value='<span style="font-size: 20px;"><b>Wavelength calculator</b></span>')

    ## Add descriptions and width for alignment
    symbol_T = pn.widgets.StaticText(value='T', width=1)
    unit_T =  pn.widgets.StaticText(value='s', width=1)
    T_widget = pn.Row(symbol_T, T, unit_T)

    symbol_h = pn.widgets.StaticText(value='h', width = 1)
    unit_h =  pn.widgets.StaticText(value='m', width = 1)
    h_widget = pn.Row(symbol_h, h, unit_h)

    symbol_L = pn.widgets.StaticText(value='L', width = 1)
    unit_L =  pn.widgets.StaticText(value='m', width = 1)
    L_widget = pn.Row(symbol_L, L, unit_L)

    # add latex formula
    text_formula = pn.widgets.StaticText(value='Solves iteratively:', width = 100)
    formula = pn.pane.LaTeX(r"$L=\frac{gT^2}{2 \pi} tanh( \frac{2 \pi h}{L})$")
    row_formula = pn.Row(text_formula, formula)

    ## merge the layout and display the result
    input_widget = pn.Column(title_input, T_widget, h_widget)
    output_widget = pn.Column(title_output, L_widget)
    horizontal_allignment = pn.Row(input_widget, output_widget)
    include_heading = pn.Column(heading,row_formula, horizontal_allignment)

    return include_heading

#wavelength_calculator()

In [10]:
def angular_frequency_calculator():
    # define widgets (with initial value for L)
    L = pn.widgets.FloatInput(value=5, step=0.01, width=100)
    h = pn.widgets.FloatInput(value=3, step=0.01, width=100)

    w = pn.widgets.FloatInput(value=dispersion(2*np.pi/L.value, h.value), step=0.001, width= 100, disabled=True)

    # change the value of L when a change in T or h is observed
    def update_L_widget(event):
        w.value = dispersion(2*np.pi/L.value, h.value)

    L.param.watch(update_L_widget, 'value')
    h.param.watch(update_L_widget, 'value')

    # set the surrounding layout, like headings and descriptions
    ## set headings and the column width
    title_input = pn.widgets.StaticText(value='<b>Input</b>', width=200)
    title_output = pn.widgets.StaticText(value='<b>Output</b>', width=200)
    heading = pn.widgets.StaticText(value='<span style="font-size: 20px;"><b>Angular frequency calculator (w)</b></span>')

    ## Add descriptions and width for alignment
    symbol_L = pn.widgets.StaticText(value='L', width=1)
    unit_L =  pn.widgets.StaticText(value='m', width=1)
    L_widget = pn.Row(symbol_L, L, unit_L)

    symbol_h = pn.widgets.StaticText(value='h', width = 1)
    unit_h =  pn.widgets.StaticText(value='m', width = 1)
    h_widget = pn.Row(symbol_h, h, unit_h)

    symbol_w = pn.widgets.StaticText(value='w', width = 1)
    unit_w=  pn.widgets.StaticText(value='m', width = 1)
    w_widget = pn.Row(symbol_w, w, unit_w)

    # add latex formula
    text_formula = pn.widgets.StaticText(value='Solves:', width = 100)
    formula = pn.pane.LaTeX(r"$\omega = \sqrt{ g k tanh(kh)}$")
    row_formula = pn.Row(text_formula, formula)
    
    ## merge the layout and display the result
    input_widget = pn.Column(title_input, L_widget, h_widget)
    output_widget = pn.Column(title_output, w_widget)
    horizontal_allignment = pn.Row(input_widget, output_widget)
    include_heading = pn.Column(heading, row_formula, horizontal_allignment)

    return include_heading

#angular_frequency_calculator()

In [11]:
def W2_calculators():
    allignment = pn.Row(wavelength_calculator(), angular_frequency_calculator())
    return allignment

#W2_calculators()

## Questions

### Q1

In [10]:
def W2_Q1():
    T1 = round(uniform(5, 8), 1)
    h1 = round(uniform(0.5, 5), 1)

    text_general = "Can you asses the wavelength through an iterative approach when the wave period (T) is " + str(T1) + " seconds and the water depth (h) is " + str(h1) + " meter? Feel free to use the cell below to make the computations."
    text_widget = pn.widgets.StaticText(value=text_general, width = 750)
    text_widget = pn.widgets.StaticText(value=text_general)
    
    Q1_text = "a) What is the deep water wavelength?"
    Q1_unit = " m"
    L = 9.81 * T1**2 / (2 * np.pi)
    Q1_answer = round(L, 2)
    Q1_FB_G = 'Indeed, the deep water wavelength is in this way related to the wave period.'
    Q1_FB_W = 'There is a mistake, the only variable is the wave period.'

    Q2_text = "b) If you use the dispersion relation expressed in the wavelength, what is the wavelength in the first iteration?"
    Q2_unit = " m"
    L = 9.81 * T1**2 / (2 * np.pi) * np.tanh(2 * np.pi * h1 / L)
    Q2_answer = round(L, 2)
    Q2_FB_G = ''
    Q2_FB_W = 'Try to fill in the deep water wavelength for the first iteration.'

    Q3_text = "c) To what wavelength does this iteration converge?"
    Q3_unit = " m"
    Q3_answer = round(wave_length(T1, h1), 2)
    Q3_FB_G = ''
    Q3_FB_W = 'There is a mistake, '

    questions = [Q1_text, Q2_text,Q3_text]
    units = [Q1_unit, Q2_unit, Q3_unit]
    answers =[Q1_answer, Q2_answer, Q3_answer]
    FB_good = [Q1_FB_G, Q2_FB_G, Q3_FB_G]
    FB_wrong = [Q1_FB_W, Q2_FB_W, Q3_FB_W]

    
    all_widgets = nummeric_question_body(questions, units, answers, FB_good, FB_wrong)
    
    display(pn.Column(text_widget,*all_widgets))
    
#W2_Q1()

### Q2

In [11]:
def W2_Q2():
    T1 = round(uniform(5, 8), 1)
    h1 = round(uniform(0.5, 5), 1)

    #T1 = 6.7
    #h1 = 4.5
    
    text_general = "Can you asses the wavelength through table B-3 of the book, when the wave period (T) is " + str(T1) + " seconds and the water depth (h) is " + str(h1) + " meter? Feel free to use the cell below to make the computations."
    text_widget = pn.widgets.StaticText(value=text_general, width = 750)
    text_widget = pn.widgets.StaticText(value=text_general)
    
    Q1_text = "a) What is the ratio of h/L0? (Use 3 decimal numbers in this question)"
    Q1_unit = ""
    L0 = 9.81 * T1**2 / (2 * np.pi)
    Q1_answer = round(h1 / L0, 3)
    Q1_FB_G = ''
    Q1_FB_W = ''

    Q2_text = "b) What is the ratio of h/L, you can linear interpolate. (Use 3 decimal numbers in this question)"
    Q2_unit = ""

    # data from table B-3, page 536, of the book CD1
    h_L0_serie = np.array(
        [
            0,
            0.002,
            0.004,
            0.006,
            0.008,
            0.01,
            0.015,
            0.02,
            0.025,
            0.03,
            0.035,
            0.04,
            0.045,
            0.05,
            0.055,
            0.06,
            0.065,
            0.07,
            0.075,
            0.08,
            0.085,
            0.09,
            0.095,
            0.1,
            0.11,
            0.12,
            0.13,
            0.14,
            0.15,
            0.16,
            0.17,
            0.18,
            0.19,
            0.2,
            0.21,
            0.22,
            0.23,
            0.24,
            0.25,
            0.26,
            0.27,
            0.28,
            0.29,
            0.30,
            0.31,
            0.32,
            0.33,
            0.34,
            0.35,
            0.36,
            0.37,
            0.38,
            0.39,
            0.4,
            0.41,
            0.42,
            0.43,
            0.44,
            0.45,
            0.46,
            0.47,
            0.48,
            0.49,
            0.5,
            1,
        ]
    )
    h_L_serie = np.array(
        [
            0,
            0.0179,
            0.0253,
            0.0311,
            0.0360,
            0.0403,
            0.0496,
            0.0576,
            0.0648,
            0.0713,
            0.0775,
            0.0833,
            0.0888,
            0.0942,
            0.0993,
            0.104,
            0.109,
            0.114,
            0.119,
            0.123,
            0.128,
            0.132,
            0.137,
            0.141,
            0.150,
            0.158,
            0.167,
            0.175,
            0.183,
            0.192,
            0.2,
            0.208,
            0.217,
            0.225,
            0.234,
            0.242,
            0.251,
            0.259,
            0.268,
            0.277,
            0.285,
            0.294,
            0.303,
            0.312,
            0.321,
            0.330,
            0.339,
            0.349,
            0.358,
            0.367,
            0.377,
            0.386,
            0.395,
            0.405,
            0.415,
            0.424,
            0.434,
            0.433,
            0.453,
            0.463,
            0.472,
            0.482,
            0.492,
            0.502,
            1,
        ]
    )

    # read the table
    id_lower = np.where(h_L0_serie <= h1 / L0)[0][
        -1
    ]  # get the (last) id where the value is below h/L0
    h_L_lower = h_L_serie[
        id_lower
    ]  # use this id to get the new ratio H/L, the lower boundary
    h_L_upper = h_L_serie[id_lower + 1]  # the upper boundary

    # linear interpolate
    slope = (h_L_upper - h_L_lower) / (h_L0_serie[id_lower + 1] - h_L0_serie[id_lower])
    h_L = h_L_lower + slope * (h1 / L0 - h_L0_serie[id_lower])

    # print(h1/L0, id_lower, h_L_lower, h_L_upper, h_L)
    Q2_answer = round(h_L, 3)
    Q2_FB_G = ''
    Q2_FB_W = ''

    Q3_text = (
        "c) What is the wavelength?"
    )
    Q3_unit = " m"
    Q3_answer = round(h1 / h_L, 2)
    Q3_FB_G = ''
    Q3_FB_W = ''

    Q4_text = "d) What is the value tanh(kh), you can linearly interpolate. (Use 3 decimal numbers in this question)"
    Q4_unit = ""

    # data from table B-3
    tanh_kh_serie = np.array(
        [
            0,
            0.112,
            0.158,
            0.193,
            0.222,
            0.248,
            0.302,
            0.347,
            0.386,
            0.420,
            0.452,
            0.48,
            0.507,
            0.531,
            0.554,
            0.575,
            0.595,
            0.614,
            0.632,
            0.649,
            0.665,
            0.681,
            0.695,
            0.681,
            0.695,
            0.709,
            0.735,
            0.759,
            0.780,
            0.8,
            0.818,
            0.835,
            0.85,
            0.864,
            0.877,
            0.888,
            0.899,
            0.909,
            0.918,
            0.926,
            0.933,
            0.940,
            0.946,
            0.952,
            0.957,
            0.961,
            0.965,
            0.969,
            0.972,
            0.975,
            0.978,
            0.980,
            0.983,
            0.984,
            0.986,
            0.988,
            0.989,
            0.990,
            0.991,
            0.992,
            0.993,
            0.994,
            0.995,
            0.995,
            0.996,
            0.996,
            1,
        ]
    )

    # read the table
    id_lower = np.where(h_L0_serie <= h1 / L0)[0][
        -1
    ]  # get the (last) id where the value is below h/L0
    tanh_kh_lower = tanh_kh_serie[id_lower]
    tanh_kh_upper = tanh_kh_serie[id_lower + 1]

    # linear interpolate
    slope = (tanh_kh_upper - tanh_kh_lower) / (
        h_L0_serie[id_lower + 1] - h_L0_serie[id_lower]
    )
    tanh_kh = tanh_kh_lower + slope * (h1 / L0 - h_L0_serie[id_lower])

    Q4_answer = round(tanh_kh, 3)
    Q4_FB_G = ''
    Q4_FB_W = ''

    questions = [Q1_text, Q2_text,Q3_text, Q4_text]
    units = [Q1_unit, Q2_unit, Q3_unit, Q4_unit]
    answers =[Q1_answer, Q2_answer, Q3_answer, Q4_answer]
    FB_good = [Q1_FB_G, Q2_FB_G, Q3_FB_G, Q4_FB_G]
    FB_wrong = [Q1_FB_W, Q2_FB_W, Q3_FB_W, Q4_FB_W]
    
    all_widgets = nummeric_question_body(questions, units, answers, FB_good, FB_wrong)
    
    display(pn.Column(text_widget,*all_widgets))

    #print(h1 / L0)
    #print(h_L)
    #print(h_L_lower)
    #print(h_L_upper)
    #print(tanh_kh_lower)
    #print(tanh_kh_upper)
    
#W2_Q2()

### Q3

In [12]:
def W2_Q3():   
    T1 = round(uniform(5, 8), 1)
    h1 = round(uniform(0.5, 5), 1)

    T1 = 6.7
    h1 = 4.5

    text_general = "A third method to calculate the wave length is through the implicit formula of Fenton, when the wave period (T) is " + str(T1) + " seconds and the water depth (h) is " + str(h1) + " meter? Feel free to use the cell below to make the computations."
    text_widget = pn.widgets.StaticText(value=text_general, width = 750)
    text_widget = pn.widgets.StaticText(value=text_general)

    def waveNumber_Fenton(T, d):  # made by J.A.ArriagaGarcia (Jaime)
        g = 9.81
        omega = 2 * np.pi / T
        k0 = omega * omega / g
        alpha = k0 * d
        beta = alpha * (np.tanh(alpha)) ** -0.5
        k = (
            (alpha + beta**2 * np.cosh(beta) ** -2)
            / (np.tanh(beta) + beta * np.cosh(beta) ** -2)
            / d
        )
        return k
    
    Q1_text = (
        "a) What is the wave number (k), following the explicit formula of Fentom?"
    )
    Q1_unit = ""

    k1 = waveNumber_Fenton(T=T1, d=h1)
    Q1_answer = round(k1, 2)
    Q1_FB_G = ''
    Q1_FB_W = ''

    Q2_text = "b) What is the wave length, following the explicit formula of Fentom?"
    Q2_unit = " m"
    Q2_answer = round(2 * np.pi / k1, 2)
    Q2_FB_G = ''
    Q2_FB_W = ''

    questions = [Q1_text, Q2_text]
    units = [Q1_unit, Q2_unit]
    answers =[Q1_answer, Q2_answer]
    FB_good = [Q1_FB_G, Q2_FB_G]
    FB_wrong = [Q1_FB_W, Q2_FB_W]
    
    all_widgets = nummeric_question_body(questions, units, answers, FB_good, FB_wrong)
    
    display(pn.Column(text_widget,*all_widgets))

#W2_Q3()

In [13]:
def nummeric_question_body(questions, units, answers, FB_good, FB_wrong, random_order = False):
    all_widgets = []
    attempts = []

    order = np.arange(0, len(questions), 1)
    if random_order == True:
        shuffle(order)

    for i in np.array(order):
        question, unit, answer, Q_FB_G, Q_FB_W = questions[i], units[i], answers[i], FB_good[i], FB_wrong[i]
        id = i+1 
        question_widget = pn.widgets.StaticText(value=question, width = 750)
        unit_widget = pn.widgets.StaticText(value=unit, width = 10)
        num_widget = pn.widgets.FloatInput(value=0, step=0.01, width = 100)
        #feedback_widget = pn.widgets.TextInput(value="", name="", width=500)
        feedback_widget = pn.widgets.StaticText(value="", name="", width=500)
        submit_button =  pn.widgets.Button(name="Check")
        
        Hbox = pn.Row(num_widget, unit_widget, submit_button, feedback_widget)       
        quiz_widget = pn.Column(question_widget, Hbox)

        all_widgets.append(quiz_widget)

        # the values for the submit button are determined at the moment these are created.
        attempt = pn.widgets.FloatInput(value=0)
        attempts.append(attempt)
        submit_button.on_click(check_nummeric_answers(id, answer, unit, Q_FB_G, Q_FB_W, num_widget, feedback_widget, attempt))
        
    return all_widgets


### Q4

In [14]:
def W2_Q4():

    h = round(uniform(2, 4), 1)  # m , random value from 2 to 4 (excluding 4)
    L = round(uniform(25, 30), 1)  # m
    k = 2 * np.pi / L
    omega = dispersion(k, h)

    Q1_text = (
        "What is the wave period (T) for a wave with a length (L) of "
        + str(L)
        + " meter when the depth (h) is "
        + str(h)
        + " meters?"
    )
    Q1_unit = " s"
    Q1_answer = T = round(2 * np.pi / omega, 2)
    Q1_FB_G = ''
    Q1_FB_W = ''

    
    h = round(uniform(2, 4), 1)  # m
    L = round(uniform(20, 30), 1)  # m
    k = 2 * np.pi / L
    omega = dispersion(k, h)
    T = 2 * np.pi / omega

    Q2_text = (
        "What is the wave celerity (c) for waves with a length (L) of "
        + str(L)
        + " meter when the depth (h) is "
        + str(h)
        + " meters?"
    )
    Q2_unit = " m/s"
    Q2_answer = c = round(L / T, 2)
    Q2_FB_G = ''
    Q2_FB_W = ''

    
    h = round(uniform(1, 2), 1)
    L = round(uniform(20, 30), 1)  # m
    omega = dispersion(k, h)
    T = 2 * np.pi / omega

    Q3_text = (
        "What is the wave celerity (c) for waves with a length (L) of "
        + str(L)
        + " meter when the depth (h) is "
        + str(h)
        + " meters?"
    )
    Q3_unit = " m/s"
    Q3_answer = c = round(L / T, 2)
    Q3_FB_G = ''
    Q3_FB_W = ''


    h = round(uniform(1, 2), 1)
    T = round(uniform(5, 7), 1)
    L = wave_length(T, h)
    Q4_text = (
        "What is the wave length (L) for a wave with a period (T) of "
        + str(T)
        + " seconds when the depth (h) is "
        + str(h)
        + " meters?"
    )
    Q4_unit = " m"
    Q4_answer = round(L, 2)
    Q4_FB_G = ''
    Q4_FB_W = ''
    

    h = round(uniform(5, 7), 1)  # km
    L = 500000  # m
    k = 2 * np.pi / L
    omega = dispersion(k, h * 1000)  # m to km
    T = 2 * np.pi / omega
    Q5_text = (
        "What is the propagation speed (c) of a tsunami wave train with (L) of 500 km in the deep ocean with a depth (h) of "
        + str(h)
        + " km?"
    )
    Q5_unit = " km/h"
    Q5_answer = c = round(L / T / 1000 * 3600, 2)
    Q5_FB_G = ''
    Q5_FB_W = ''

    
    h = round(uniform(5, 7), 1)  # km
    T = round(uniform(30, 38), 0)  # min
    L = wave_length(T * 60, h)
    Q6_text = (
        "What is the propagation speed of a tsunami wave train with a period (T) of "
        + str(T)
        + " minutes in the deep ocean with a depth (h) of "
        + str(h)
        + " km?"
    )
    Q6_unit = " km/h"
    Q6_answer = c = round(L / (T) / 1000 * 60, 2)
    Q6_FB_G = ''
    Q6_FB_W = ''

    
    h = round(uniform(0.6, 10), 1)
    T = round(uniform(2.5, 12), 1)
    L = round(wave_length(T, h), 1)  # releastic value for question, based on T

    k = 2 * np.pi / L
    n = 0.5 + k * h / np.sinh(2 * k * h)
    Q7_text = (
        "What is the value of n when the water depth (h) is "
        + str(h)
        + " m, and the wave length (L) is "
        + str(L)
        + " m?"
    )
    Q7_unit = ""
    Q7_answer = round(n, 2)
    Q7_FB_G = ''
    Q7_FB_W = ''
    

    h = round(uniform(0.8, 10), 1)
    T = round(uniform(2.5, 12), 1)
    L = wave_length(T, h)
    k = 2 * np.pi / L
    c = L / T
    n = 0.5 + k * h / np.sinh(2 * k * h)
    cg = n * c

    Q8_text = (
        "What is the group velocity (cg) when the water depth (h) is "
        + str(h)
        + " m, and the wave period (T) is "
        + str(T)
        + " s?"
    )
    Q8_unit = " m/s"
    Q8_answer = round(cg, 2)
    Q8_FB_G = ''
    Q8_FB_W = ''

    
    # get realistic value
    h = round(uniform(1, 10), 1)
    T = round(uniform(4, 8), 1)
    L = round(wave_length(T, h), 1)

    omega = dispersion(k, h)
    T = 2 * np.pi / omega
    k = 2 * np.pi / L
    c = L / T
    n = 0.5 + k * h / np.sinh(2 * k * h)
    cg = n * c

    Q9_text = (
        "At which speed does the wave energy propagate when the water depth (h) is "
        + str(h)
        + " m, and the wave length (L) is "
        + str(L)
        + " m?"
    )
    Q9_unit = " m/s"
    Q9_answer = round(cg, 2)
    Q9_FB_G = ''
    Q9_FB_W = ''

    questions = [Q1_text, Q2_text,Q3_text, Q4_text, Q5_text, Q6_text, Q7_text, Q8_text, Q9_text]
    units = [Q1_unit, Q2_unit, Q3_unit, Q4_unit, Q5_unit, Q6_unit, Q7_unit, Q8_unit, Q9_unit]
    answers =[Q1_answer, Q2_answer, Q3_answer, Q4_answer, Q5_answer, Q6_answer, Q7_answer, Q8_answer, Q9_answer]
    FB_good = [Q1_FB_G, Q2_FB_G, Q3_FB_G, Q4_FB_G, Q5_FB_G, Q6_FB_G, Q7_FB_G, Q8_FB_G, Q9_FB_G]
    FB_wrong = [Q1_FB_W, Q2_FB_W, Q3_FB_W, Q4_FB_W, Q5_FB_W, Q6_FB_W, Q7_FB_W, Q8_FB_W, Q9_FB_W]

    all_widgets = nummeric_question_body(questions, units, answers, FB_good, FB_wrong, random_order = True)
    
    display(pn.Column(*all_widgets))

#W2_Q4()

### Q5

In [15]:
def W2_Q5():
    h1 = uniform(0.5, 2)
    h2 = uniform(0.5, 3.5)
    h3 = uniform(0.5, 5)
    h4 = uniform(2, 5)
    h5 = uniform(0.3, 0.91)

    L1 = h1 / uniform(0.025, 0.05)  # shallow water
    L2 = h2 / uniform(0.051, 0.49)  # intermediate water
    L3 = h3 / uniform(0.5, 0.75)  # deep water
    L4 = h4 / uniform(0.025, 0.075)  # shallow or intermediate water
    L5 = h5 / uniform(0.25, 0.75)  # intermediate water or deep water

    L_serie = [L1, L2, L3, L4, L5]
    h_serie = [h1, h2, h3, h4, h5]

    # The possible answers (may very per question)
    answers = ["Shallow", "Intermediate", "Deep"]

    Questions = []
    correct_answers_id = []

    for L, h in zip(L_serie, h_serie):
        L = round(L, 2)
        h = round(h, 2)
        Questions.append("L = " + str(L) + " m, h = " + str(h) + " m")

        ratio = h / L

        if ratio <= 0.05:
            correct_answers_id.append(0)
        if ratio > 0.05 and ratio < 0.5:
            correct_answers_id.append(1)
        if ratio >= 0.5:
            correct_answers_id.append(2)

    # define the widgets for visualization, make for each a row with question and answer
    # Store all the rows with statements in a list for visualization, and in toggle_widgets for checking the answer
    Rows = []
    toggle_widgets = []
    for i in range(len(correct_answers_id)):
        question_widget = pn.widgets.StaticText(value=Questions[i], width = 150)#statement
        
        radio_group_widget = pn.widgets.RadioButtonGroup(name='Radio Button Group', options=answers, button_type='default')
        toggle_widgets.append(radio_group_widget)
        
        add_row = pn.Row(question_widget, radio_group_widget)
        Rows.append(add_row)

    # randomize the order of statements
    shuffle(Rows)

    # add a submit button with a feedback option next to it
    submit_button =  pn.widgets.Button(name="Check")
    feedback_widget = pn.widgets.StaticText(value="", name="", width=500)
    submit_row = pn.Row(submit_button, feedback_widget)
    
    # include a question
    text_general = "Select if the wave described on the left experiences shallow, intermediate, or deep water."
    text_widget = pn.widgets.StaticText(value=text_general)

    # structure the widgets and display thems
    display(text_widget, *Rows, submit_row)

    # check the answer and give feedback
    def check_answers(button):
        score = 0

        for i in range(len(correct_answers_id)):
            if toggle_widgets[i].value == answers[correct_answers_id[i]]:
                score += 1

        # print(toggle_widget.value)
        feedback_widget.value = "Your score is " + str(score) + "/" + str(len(correct_answers_id))

    submit_button.on_click(check_answers)

#W2_Q5()

### Q6

In [18]:
#from bokeh.plotting import figure
#from bokeh.layouts import column, row
#from bokeh.models import ColumnDataSource, Slider, TextInput
#from bokeh.plotting import figure, show

In [19]:
from matplotlib.figure import Figure

In [20]:
def relation_to_omega(h, w1, w2):  # T1 [s] (higher omega) < T2 [s]
    T_serie = []
    L_serie = []
    k_serie = []
    c_serie = []
    n_serie = []
    cg_serie = []
    omega_serie = np.linspace(w1, w2, 100)
    for omega in omega_serie:
        T = 2 * np.pi / omega
        L = wave_length(T, h)
        k = 2 * np.pi / L
        c = L / T
        n = 0.5 + k * h / np.sinh(2 * k * h)
        cg = n * c

        T_serie.append(T)
        L_serie.append(L)
        k_serie.append(k)
        c_serie.append(c)
        n_serie.append(n)
        cg_serie.append(cg)

    return omega_serie, T_serie, L_serie, k_serie, c_serie, n_serie, cg_serie

# ans = relation_to_omega(h = 5, T1 = 2, T2 = 60) # works for T1 = 2 sec, T2 = 60 sec, h = 5 m
ans = relation_to_omega(h=6000, w1= 2*np.pi/ (60*5), w2= 2*np.pi/(60 * 60))  # works for T1 = 2 sec, T2 = 60 sec, h = 5 m

In [55]:
def relation_to_omega(h, w1, w2):  # T1 [s] (higher omega) < T2 [s]
    omega_serie = np.linspace(w1, w2, 100)
    T_serie = [2 * np.pi/w for w in omega_serie]
    L_serie = [wave_length(T, h) for T in T_serie]
    k_serie = [2*np.pi/L for L in L_serie]
    c_serie = [L/T for L,T in zip(L_serie, T_serie)]
    n_serie = [0.5 + k*h/np.sinh(2*k*h) for k in k_serie]
    cg_serie = [c*n for c,n in zip(c_serie, n_serie)]

    return omega_serie, T_serie, L_serie, k_serie, c_serie, n_serie, cg_serie

In [61]:
def check_answers_W2_Q6(id, answers, unit, FB_G, FB_W, num_widgets, feedback_widgets, attempt, final_score_widget, plots, panes, figures, par):
    FV = par
    omega_serie, T_serie, L_serie, k_serie, c_serie, n_serie, cg_serie = FV['omega_serie'], FV['T_serie'], FV['L_serie'], FV['k_serie'], FV['c_serie'], FV['n_serie'], FV['cg_serie']
    
    def button_callback(b):
        attempt.value += 1

        # store responses in a list for plotting
        responses = []
        score = 0
        for i in range(len(answers)):
            num_widget = num_widgets[i]
            feedback_widget = feedback_widgets[i]
            answer = answers[i]
            response = num_widget.value
            responses.append(response)

            # if answer is correct
            if np.abs(answer - num_widget.value) < limit_answer(answer):
                score += 1
                if len(FB_G) != 0:
                    feedback_widget.value = FB_G
                else:
                    feedback_widget.value = 'Well done, this is correct!'

            # the answer is NOT within boundaries, provide feedback based on the number of attempts
            if np.abs(answer - num_widget.value) >= limit_answer(answer):
    
                if attempt.value < 3 and len(FB_W) > 0:
                    feedback_widget.value = FB_W
                    
                if attempt.value < 3 and len(FB_W) == 0:
                    feedback_widget.value = 'Oops, there seems to be a mistake'
                    
                if attempt.value >= 3:
                    feedback_widget.value = 'The correct answer is ' + str(answer) + str(unit) + '.'

        final_score_widget.value = str(score) + '/' + str(len(answers))

        # Update the graphs
        if id == 1:
            ax = plots[0]
            ax.clear()
            line = ax.plot(omega_serie, L_serie, label = 'L [m]')
            #ax.scatter(par['omega_points'], np.array(answers) * 1000, c="green", s=30, label = 'correct answer')
            ax.scatter(par['omega_points'], np.array([responses]) * 1000, c="#006AB5", s=25, label = 'answer')
            ax.set_title(par['titles'][0])
            ax.legend()
            ax.set_xlabel("$\omega $ [rad/s]")
            panes[0].object = figures[0]
            
        if id == 2 or id == 3:
            # the number of times question 2 and 3 are answered
            attempt2 = par['attempts'][1].value
            attempt3 = par['attempts'][2].value

            # the given answers
            responses2 = return_answers_widgets(par['all_answers'][1])
            responses3 = return_answers_widgets(par['all_answers'][2])

            ax = plots[1]
            ax.clear()
            if attempt2 > 0:
                line = ax.plot(omega_serie, c_serie, label = 'c [m/s]', color = '#006AB5')
                #ax.scatter(par['omega_points'], answers, color ="#006AB5", s=30, label = 'correct answer')
                ax.scatter(par['omega_points'], responses2, color="#006AB5", s=25, label = 'answer c')
                ax.set_title(par['titles'][1])
                ax.legend()
                ax.set_xlabel("$\omega $ [rad/s]")
                panes[1].object = figures[1]

            if attempt3 > 0:
                line = ax.plot(omega_serie, cg_serie, label = 'cg [m/s]', color = '#003f6c')
                ax.scatter(par['omega_points'], responses3, c="#003f6c", s=25, label = 'answer cg')
                ax.set_title(par['titles'][1])
                ax.legend()
                ax.set_xlabel("$\omega $ [rad/s]")
                panes[1].object = figures[1]

        if id == 3:
            ax = plots[2]
            ax.clear()
            line = ax.plot(omega_serie, n_serie, label = 'n [-]')
            ax.set_title(par['titles'][2])
            ax.legend()
            ax.set_xlabel("$\omega $ [rad/s]")
            panes[2].object = figures[2] 

    return button_callback  # otherwise gives TypeError: 'NoneType' object is not callable

In [62]:
def return_answers_widgets(widgets):
    answers = []
    for widget in widgets:
        answers.append(widget.value)
    return answers

In [140]:
def W2_Q6():
    h = np.arange(3000, 6000 + 500, 500)
    h = np.random.choice(h)

    T1 = 5 * 60
    T4 = 30 * 60

    text_general = "We are going to analyze the characteristics of tsunami waves at a depth of " + str(h) + " m. It will be analyzed for waves with a period of "  + str(T1/60) + " minutes (point 1), at the shallow water boundary (point 2), the deep water boundary (point 3), and for waves with a period of " + str(T4/60) + "  minutes (point 4)."

    Q1 = "What is the wave length of the tsunami wave at those 4 points?"
    Q1_labels = ["L1", "L2", "L3", "L4"]
    Q1_unit = " km"

    L1 = wave_length(T=T1, h=h)
    L2 = h / 0.05
    L3 = h / 0.5
    L4 = wave_length(T=T4, h=h)
    Q1_answers = [round(L1 / 1000, 2), L2 / 1000, L3 / 1000, round(L4 / 1000, 2)]

    # The 4 points of interest in this question
    w1 = 2 * np.pi / T1
    w2 = dispersion(k=2 * np.pi / (L2), h=h)
    w3 = dispersion(k=2 * np.pi / (L3), h=h)
    w4 = 2 * np.pi / T4
    omega_points = np.array([w1, w2, w3, w4])

    Q2 = "What is the wave celerity (c) of the tsunami wave at those 4 points?"
    Q2_labels = ["c1", "c2", "c3", "c4"]
    Q2_unit = " m/s"
    T2 = 2 * np.pi / w2
    T3 = 2 * np.pi / w3
    
    def calc_c(T,h):
        L = wave_length(T, h)
        return 9.81*T/(2*np.pi)*np.tanh(2*np.pi*h/L)

    c1 = L1 / T1
    c2 = L2 / T2
    c3 = L3 / T3
    c4 = L4 / T4

    c1 = calc_c(T1,h)
    c2 = calc_c(T2,h)
    c3 = calc_c(T3,h)
    c4 = calc_c(T4,h)
    
    Q2_answers = [round(c1, 2), round(c2, 2), round(c3, 2), round(c4, 2)]

    Q3 = "What is the wave group velocity (cg) of the tsunami at those 4 points?"
    Q3_labels = ["c_{g1}$", "c_{g2}", "c_{g3}", "c_{g4}"]
    Q3_unit = " m/s"

    k1 = 2 * np.pi / L1
    k2 = 2 * np.pi / L2
    k3 = 2 * np.pi / L3
    k4 = 2 * np.pi / L4

    n1 = 0.5 + k1 * h / np.sinh(2 * k1 * h)
    n2 = 0.5 + k2 * h / np.sinh(2 * k2 * h)
    n3 = 0.5 + k3 * h / np.sinh(2 * k3 * h)
    n4 = 0.5 + k4 * h / np.sinh(2 * k4 * h)

    cg1 = c1 * n1
    cg2 = c2 * n2
    cg3 = c3 * n3
    cg4 = c4 * n4

    Q3_answers = [round(cg1, 2), round(cg2, 2), round(cg3, 2), round(cg4, 2)]

    # store the questions in a list
    Questions = [Q1, Q2, Q3]
    Unit_question = [Q1_unit, Q2_unit, Q3_unit]
    answer_question = [Q1_answers, Q2_answers, Q3_answers]
    label_question = [Q1_labels, Q2_labels, Q3_labels]

    # no feedback provided
    FB_G, FB_W = '', ''

    def calc_c(T,h):
        L = wave_length(T, h)
        return 9.81*T/(2*np.pi)*np.tanh(2*np.pi*h/L)
    
    # the answers for the graph
    omega_serie = np.linspace(w4, w3*2, 100)
    T_serie = [2 * np.pi/w for w in omega_serie]
    L_serie = [wave_length(T, h) for T in T_serie]
    k_serie = [2*np.pi/L for L in L_serie]
    #c_serie = [L/T for L,T in zip(L_serie, T_serie)]
    c_serie = [calc_c(T,h) for T in T_serie]
    n_serie = [0.5 + k*h/np.sinh(2*k*h) for k in k_serie]
    cg_serie = [c*n for c,n in zip(c_serie, n_serie)]

    #set plot settings and make plots   
    titles = ['Wave length', 'Celerity', 'n']
    figures = []
    plots = []
    panes = []
    for i, (title) in enumerate(titles):
        fig = Figure((5,2.5))
        ax = fig.subplots()
        ax.set_axis_off()
        pane = pn.pane.Matplotlib(fig, dpi=96)
        fig.subplots_adjust(hspace=0)
        fig.subplots_adjust(bottom=0.25)  # Add some extra space for the axis at the bottom
        fig.subplots_adjust(left=0.2)  # Add some space for the labels
        figures.append(fig)
        plots.append(ax)
        panes.append(pane)
    
    # fill an empty list with widgets
    all_question_widgets = []
    all_answers = []
    attempts = [] # widgets only used for counting how many times the submit button is pressed
    id = 0  
    for i, (question, unit, answers, label) in enumerate(zip(Questions, Unit_question, answer_question, label_question)):
        id += 1
        question_widget = pn.widgets.StaticText(value=question, width = 750)

        Rows_answer = []
        num_widgets = []
        feedback_widgets = []
        for answer in answers:
            unit_widget = pn.widgets.StaticText(value=unit, width = 10)
            num_widget = pn.widgets.FloatInput(value=0, step=0.01, width = 100)
            feedback_widget = pn.widgets.StaticText(value="", name="", width=250)

            num_widgets.append(num_widget)
            feedback_widgets.append(feedback_widget)
            
            Hbox = pn.Row(num_widget, unit_widget, feedback_widget)
            Rows_answer.append(Hbox)
        all_answers.append(num_widgets)

        # Add a submit button with widget that returns the final score
        submit_button =  pn.widgets.Button(name="Check")
        submit_text_widget = pn.widgets.StaticText(value='Final score:', width = 70)
        final_score_widget = pn.widgets.TextInput(value="", name="", width=130)
        row_submit = pn.Row(submit_button,submit_text_widget,final_score_widget)

        # The values for the submit button are determined at the moment these are created.
        attempt = pn.widgets.FloatInput(value=0)
        attempts.append(attempt)

        # store the local parameters to be used inside the function
        FV = locals()
        submit_button.on_click(check_answers_W2_Q6(id, answers, unit, FB_G, FB_W, num_widgets, feedback_widgets, attempt, final_score_widget, plots, panes, figures, FV))

        # Structure the widgets
        Vbox_answer = pn.Column(*Rows_answer, row_submit)
        Hbox_plot = pn.Row(Vbox_answer, panes[i])
        question_widget = pn.Column(question_widget, Hbox_plot)
        all_question_widgets.append(question_widget)

    text_widget = pn.widgets.StaticText(value=text_general)
    quiz_widget = pn.Column(text_widget, *all_question_widgets)

    #print(w1, w2, w3, w4)

    return quiz_widget
    
#W2_Q6()

### Q7

In [121]:
from ipywidgets import interact, HBox, VBox

In [158]:
def dispersion(k, h):
    return (9.81 * k * np.tanh(k * h)) ** 0.5

def wave_celerity(T1, T2, T3, slope_in, d0):
    %matplotlib inline
    # bed profile
    
    slope = 1.0 / slope_in  # bed slope [-]
    d0 # offshore water depth [m]
    x_max = round((d0+2)/slope)
    x = np.arange(0, x_max + 1, 1)  # cross-shore coordinate [m]
    zbed = -(d0 - slope * x)  # bed elevation [m]
    h = -zbed  # still water depth [m]
    h[h < 0] = 0  # no negative depths

    # w is zero when h is 0, causing a divide by zero.
    # shorten the lists if a water depth of 0 is reached.
    x0_id = np.argwhere(h == 0)[0][0]  # first location where water depth = 0
    h_water = h[0:x0_id]
    x_water = x[0:x0_id]  

    # wave length through profile
    L1 = [wave_length(T1, h) for h in h_water]
    L2 = [wave_length(T2, h) for h in h_water]
    L3 = [wave_length(T3, h) for h in h_water]
    
    # velocity profile
    def calc_c(T,h):
        L = wave_length(T, h)
        return 9.81*T/(2*np.pi)*np.tanh(2*np.pi*h/L)
  
    c1 = [calc_c(T1,h) for h in h_water]
    c2 = [calc_c(T2,h) for h in h_water]
    c3 = [calc_c(T3,h) for h in h_water]

    fig, axs = plt.subplots(nrows = 3, ncols = 1, figsize = (9,6), sharex=True, sharey = False)
    fig.subplots_adjust(hspace=0)
    fig.subplots_adjust(wspace=0.1)

    # bathymetry
    axs[0].plot(x, zbed, label="Bed (1:" + str(round(slope_in,2)) + ')', color="k")
    axs[0].plot([0, x[x0_id]], [0, 0], color="gray", label="Still water surface")
    axs[0].set_ylabel("y [m]")
    axs[0].legend(loc="lower right")

    # wave length
    axs[1].plot(x_water, L1, label="wave 1")
    axs[1].plot(x_water, L2, label="wave 2")
    axs[1].plot(x_water, L3, label="wave 3")
    axs[1].set_ylim(0, np.max(([L1], [L2], [L3]))*1.1)
    axs[1].set_ylabel("Wavelength (L) [m]")
    axs[1].legend(loc="upper right")

    # wave celerity
    axs[2].plot(x_water, c1, label="wave 1")
    axs[2].plot(x_water, c2, label="wave 2")
    axs[2].plot(x_water, c3, label="wave 3")
    axs[2].set_xlim(0, np.max(x))
    axs[2].set_ylim(0, np.max(([c1], [c2], [c3]))*1.1)
    axs[2].set_ylabel("Celerity (c) [m/s]")
    axs[2].legend(loc="upper right")
    axs[2].set_xlabel('cross-shore location (x) [m]')

    # remove the lines related to the x-ticks
    axs[0].xaxis.set_visible(False)
    axs[1].xaxis.set_visible(False)

    # set title
    axs[0].set_title("Wave characteristics in cross-shore direction")

    L_max = np.max([L3, L2,L1])
    h_L = h_water/L_max
    print(h_L)
    if h_L[0] > 0.5:
        id_deep = np.argwhere(h_L <= 0.5)[0][0]  # first location where water depth = 0
        axs[1].plot(x_water[id_deep], L3[id_deep], 'ro', label="wave 3")

    id_shl = np.argwhere(h_L < 0.05)[0][0]  # first location where water depth = 0
    axs[2].plot(x_water[id_shl], c3[id_shl], 'ro', label="wave 3")

def W2_Q7():
    # Create interactive widgets, which requires IPY Widgets, widgets from panel do not work
    #L1 = pn.widgets.FloatSlider(name='Float Slider', start=0, end=3.141, step=0.01, value=1.57)
    T1 = ipw.FloatSlider(value=4, min=0.01, max=50, step=0.01, description="T1 [s]")
    T2 = ipw.FloatSlider(value=6, min=0.01, max=50, step=0.01, description="T2 [s]")
    T3 = ipw.FloatSlider(value=8, min=0.01, max=50, step=0.01, description="T3 [s]")

    slope = ipw.FloatSlider(value=30, min=0.1, max=50, step=0.1, description="slope 1:...")
    d0 = ipw.FloatSlider(value=50, min=0.1, max=500, step=0.1, description="d(x=0)")

    # Setup widget layout (User Interface) for the graph input
    vbox1 = ipw.VBox([ipw.Label('Wave components', layout=ipw.Layout(align_self='center')),T1, T2, T3])
    vbox2 = ipw.VBox([ipw.Label('General', layout=ipw.Layout(align_self='center')),slope,d0])
    UI = ipw.HBox([vbox1, vbox2])
    
    # Use the interactive function to update the plot
    graph = ipw.interactive_output(wave_celerity, {'T1': T1,'T2':T2, 'T3': T3, 'slope_in': slope, 'd0': d0})
    display(UI, graph)

W2_Q7();

Output()